# Model Improvements

This notebook covers the accuracy improvement tasks mentioned in the **optional project rubrics**. I will describe few of the experiments I did to improve upon the Faster R-CNN model which was trained in the basic model section. I followed the following approach for this:
1. Read the tensorboard output from its training and make a hypothesis for the loss or accuracy metric
2. Make the required changes and retrain the model from scratch
3. Check if the intended metrics got improved or not
4. Repeat till the resources get over :(

## 1. More Training Steps

The first and easiest change that I tried was to train the model for more number of steps since the losses in the base model were showing downward trend till 2000 epochs. So, I trained the model for 4000 steps and compared the results.

### Hypothesis

Training the model should bring down the losses further and help model perform better than the base model.

### Changes

I changed the number of training steps from **2000** to **4000**. The learning rate scheduling and other hyper params were kept as they were.

### Outcome

* Training losses went down as expected.
* Model started overfitting and performed worse in precision and recall metrics.

**Precision Comparision**

![mAP Comparision](./figures/steps_precision.png "mAP")

**Recall Comparision**

![Recall Comparision](./figures/steps_recall.png "Recall")

**Losses Comparision**

![Loss Comparision](./figures/steps_loss.png "Loss")

---

## 2. Image Augmentation

### Hypothesis

As I've mentioned in the exploratory data analysis write-up too that while exploring the type of scenes in the validation segments, I came across a few scenarios where:

1. Sun caused video segment to have a super bright region and some glares on the camera
2. Night time videos causing everything to look very dark

Although training set also had very few scenes like this, I made a hypothesis that increasing such scenes using **brightness** augmentation should help the model to perform better since self-driving cars surely need to drive in such conditions some of the time.

### Changes

I added the below **random adjust brightness** augmentation with a max delta of 0.6 (which might have been too large now that I think of it) in the `faster_rcnn_pipeline.config` so that the images can replicate sunny and night scenes better.

```
data_augmentation_options {
    random_adjust_brightness {
      max_delta: 0.6
    }
}
```

### Outcome

I trained the model in the exact similar way as the basic trained model and compared the tensorboard metrics of the two.

* The model where images were augmented performed slightly worse than the base model in all of the metrics.
* My assumption is that it happened as both models were trained for same number of steps and the newer model had more variety of images to train on. So training it for more steps should make it better than the base model.
* I didn't re-train it for more steps due to resource constraints for now. Will explore this later on.

**Precision Comparision**

![mAP Comparision](./figures/aug_precision.png "mAP")

**Recall Comparision**

![Recall Comparision](./figures/aug_recall.png "Recall")

**Losses Comparision**

![Loss Comparision](./figures/aug_loss.png "Loss")

---

## 3. Anchor Ratio Distribution

The idea was that since the Region Proposal Network losses were adding a bigger amount to the total loss value, improvements around the anchors generation should help in improving the losses.

So I planned on performing 2 steps:

1. From the given training and validation data, plot a chart of distribution of aspect ratio (width to height ratios) of the ground truth boxes.
2. Update the `first_stage_anchor_generator.grid_anchor_generator.aspect_ratios` value in the pipeline config.

**Training Bounding Boxes aspect ratio**

<img src="./figures/training_bbox_dist.png" alt="drawing" width="600"/>

**Validation Bounding Boxes aspect ratio**

<img src="./figures/validation_bbox_dist.png" alt="drawing" width="600"/>

After analyzing the graphs, I came to this conclusion that both the training and validation sets have very similar distribution of bbox ratios. And putting efforts in this direction might help with the losses but won't help with the overfitting. So I didn't go forward in this direction.

---

## Further Plans

* Experiments with the optimizer needs to be done. This include trying different types of optimizers as well as different learning rates and annealing techniques on them. This should help in stabilizing the losses.
* Spend some more time in figuring out what scenarios is the model not working well on.
* Figuring out how to run evaluation after every step/epoch to figure out the trend and if overfitting is actually happening
* Re-create the training and validation data with bigger dimensions and experiment with models with higher mAP and higher required image resolution.